In [ ]:
# Importer les bibliothèques nécessaires
import numpy as np
import pandas as pd
import os
import random
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping


In [ ]:
# Configuration des chemins
data_dir = '../data/train'

In [ ]:
# Générer les données
data = ImageDataGenerator(validation_split=0.20)

train_ds = data.flow_from_directory(data_dir,
                                    class_mode="categorical",
                                    target_size=(229, 229),
                                    batch_size=32,
                                    shuffle=True,
                                    subset='training',
                                    seed=42)

validation = data.flow_from_directory(data_dir,
                                      class_mode="categorical",
                                      target_size=(229, 229),
                                      batch_size=32,
                                      shuffle=True,
                                      subset='validation',
                                      seed=42)


In [ ]:
# Construire le modèle
IMG_SIZE = (229, 229)
base_model = EfficientNetB0(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE, 3))
base_model.trainable = False

model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(3, activation='softmax'))

model.compile(optimizer=optimizers.Adam(learning_rate=0.001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
# Afficher le résumé du modèle
model.summary()


In [ ]:
# Définir EarlyStopping
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True, min_delta=0.001, verbose=1)


In [ ]:
# Entraîner le modèle
history = model.fit(train_ds, epochs=10, validation_data=validation, callbacks=[early_stopping])


In [ ]:
# Sauvegarder le modèle entraîné
model.save('../model/model_final.h5')